In [1]:
import torch
from torch import nn
import torchvision
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

import random

import numpy as np

import matplotlib.cm as cm

import pickle

import os, sys

from denoising_assessment_project.utils import functions

# parameters setup 
from denoising_assessment_project.global_vars import global_vars

%matplotlib notebook

import os

In [2]:
# restoring variables from memory (GT images, indices_eval)
%store -r

In [3]:
# opening GT images 
with open('/Users/miriamczech/Desktop/denoising_assessment/pickled_data/EVAL/un_noised_images_eval_beam_zoom.pkl', 'rb') as un_noised_images_eval_beam_pickled:
    un_noised_images = pickle.load(un_noised_images_eval_beam_pickled)

In [ ]:
# collecting N2V-predicted images into a tensor, ensuring appropriate image loactions relative to GT images
import glob
import fnmatch
from tifffile import imread

#path at which the predicted images are stored 
path = '/Users/miriamczech/Desktop/denoising_assessment/tiffed_data_beam/predictions/N2V_CPU_run'

predicted_images = os.listdir(path)
predicted_images_len = len(predicted_images)

predicted_images_tensor = torch.zeros_like(un_noised_images)
for n in range(0,20):
    noised_imgs_n_list = glob.glob('/Users/miriamczech/Desktop/denoising_assessment/tiffed_data_beam/predictions/N2V_CPU_run/{}_*_*.tif'.format(n))
    for m in range(0,20):
        noised_imgs_m_list = fnmatch.filter(noised_imgs_n_list, '/Users/miriamczech/Desktop/denoising_assessment/tiffed_data_beam/predictions/N2V_CPU_run/{}_{}_*.tif'.format(n, m))
        for i in range(0,50):
            img_i_path = fnmatch.filter(noised_imgs_m_list, '/Users/miriamczech/Desktop/denoising_assessment/tiffed_data_beam/predictions/N2V_CPU_run/{}_{}_{}.tif'.format(n,m,i))[0]
            #print(img_i_path)
            xy = torch.tensor(imread(img_i_path))
            predicted_images_tensor[n,m,i,:,:] = xy
            

In [ ]:
# storing the predicted_images_tensor
predicted_images_N2V_tensor_beam_pickled = pickle.dump(predicted_images_tensor, open("/Users/miriamczech/Desktop/denoising_assessment/pickled_data/EVAL/predicted_images_N2V_tensor_cpu.pkl", 'wb'), protocol=4)
%store predicted_images_N2V_tensor_beam_pickled

In [ ]:
# CARE: -NRMSE-
# 4a. Computing NRMSE for each GT-CARE_prediction image pair in the data collection
# dims=[m, n, reps]
metric_output = functions.NRMSE_compute(un_noised_images, predicted_images_tensor)

# 5. Computing statistics for the metric across all n x m sets 
# dims=[m, n, 2]
metric_stats_NRMSE_N2V = functions.set_mean_std_compute(metric_output)

In [ ]:
# CARE: -RMSE-
# 4b. Computing RMSE for each image in the data collection
# dims=[m, n, reps]
metric_output = functions.RMSE_compute(un_noised_images, predicted_images_tensor)

#tests 
print(metric_output.shape)

# 5. Computing statistics for the metric across all n x m sets 
# dims=[m, n, 2]
metric_stats_RMSE_N2V = functions.set_mean_std_compute(metric_output)

In [ ]:
# CARE: -PSNR- 
# 4c. Computing PSNR for each image in the data collection
# dims=[m, n, reps]
metric_output = functions.PSNR_compute(un_noised_images, predicted_images_tensor)

#tests 
print(metric_output.shape)

# 5. Computing statistics for the metric across all n x m sets 
# dims=[m, n, 2]
metric_stats_PSNR_N2V = functions.set_mean_std_compute(metric_output)

In [ ]:
# CARE: -SSIM- 
# 4c. Computing SSIM for each image in the data collection
# dims=[m, n, reps]
metric_output = functions.SSIM_compute(un_noised_images, predicted_images_tensor)
#tests 
print(metric_output.shape)

# 5. Computing statistics for the metric across all n x m sets 
# dims=[m, n, 2]
metric_stats_SSIM_N2V = functions.set_mean_std_compute(metric_output)

In [ ]:
# CARE: -MS_SSIM-
# 4c. Computing MS_SSIM for each image in the data collection
# dims=[m, n, reps]
metric_output = functions.MS_SSIM_compute(un_noised_images, predicted_images_tensor)
#tests 
print(metric_output.shape)

# 5. Computing statistics for the metric across all n x m sets 
# dims=[m, n, 2]
metric_stats_MS_SSIM_N2V = functions.set_mean_std_compute(metric_output)

In [ ]:
%store metric_stats_NRMSE_N2V
%store metric_stats_RMSE_N2V
%store metric_stats_PSNR_N2V
%store metric_stats_SSIM_N2V
%store metric_stats_MS_SSIM_N2V

In [ ]:
# CARE: -NRMSE-
# 6. plotting obtained stats
functions.plot_stats(metric_stats_NRMSE_N2V, indices_eval, 'N2V - NRMSE')

In [ ]:
# CARE: -RMSE-
# 6. plotting obtained stats
functions.plot_stats(metric_stats_RMSE_N2V, indices_eval, 'N2V - RMSE')

In [ ]:
# CARE: -PSNR-
# 6. plotting obtained stats
functions.plot_stats(metric_stats_PSNR_N2V, indices_eval, 'N2V - PSNR')

In [ ]:
# CARE: -SSIM-
# 6. plotting obtained stats
functions.plot_stats(metric_stats_SSIM_N2V, indices_eval, 'N2V - SSIM')

In [ ]:
# CARE: -MS_SSIM-
# 6. plotting obtained stats
functions.plot_stats(metric_stats_MS_SSIM_N2V, indices_eval, 'N2V - MS_SSIM')